In [1]:
from usda_mmn_utils import *


In [2]:
markets_df = get_markets_list()
print(markets_df.head())


  slug_id                  market_name
0    2277               Atlanta fruits
1    2278           Atlanta vegetables
2    2279  Atlanta onions and potatoes
3    2280                 Atlanta nuts
4    2281             Baltimore fruits


In [3]:
commodities_list = get_commodities_list(slug_id = markets_df.iloc[0]['slug_id'])
print(commodities_list)


['Limes', 'Lemons', 'Grapes', 'Plantains', 'Coconuts', 'Apples', 'Papaya', 'Bananas', 'Peaches', 'Kiwifruit', 'Mangoes', 'Oranges', 'Guava', 'Grapefruit', 'Avocados', 'Cantaloupes', 'Blueberries', 'Raspberries', 'Pears', 'Strawberries', 'Blackberries', 'Honeydews', 'Pineapples', 'Dates', 'Watermelons', 'Carambola - Star Fruit', 'Rambutan']


In [4]:
prices_df = get_prices(commodity=commodities_list[0], slug_id=markets_df.iloc[0]['slug_id'], start_year=2023)

In [5]:
print(prices_df.head())

  report_date  slug_id commodity        variety        package item_size  \
0  12/29/2023     2277     Limes  SEEDLESS TYPE  40 lb cartons      200s   
1  12/29/2023     2277     Limes  SEEDLESS TYPE  40 lb cartons      150s   
2  12/29/2023     2277     Limes  SEEDLESS TYPE  10 lb cartons       54s   
3  12/29/2023     2277     Limes  SEEDLESS TYPE  10 lb cartons       48s   
4  12/29/2023     2277     Limes  SEEDLESS TYPE  40 lb cartons      150s   

  properties           grade organic     origin low_price high_price  \
0       None  No Grade Marks       N     Mexico     31.00      34.50   
1       None  No Grade Marks       N     Mexico     33.00      36.00   
2       None  No Grade Marks       N     Mexico     11.50      13.00   
3       None  No Grade Marks       N     Mexico     11.50      11.50   
4       None  No Grade Marks       N  Guatemala     25.00      25.00   

  unit_sales  
0        N/A  
1        N/A  
2        N/A  
3        N/A  
4        N/A  
